In [ ]:
!pip install gym
!pip install stable-baselines3 --quiet
!pip install 'shimmy>=0.2.1' --quiet
!pip install optuna --quiet
!pip install tslearn --quiet
!pip install pykrx --quiet
!pip install finance-datareader --quiet
!pip install chardet
!pip install seaborn
!pip install yfinance
!pip install openpyxl
!pip install optuna --quiet
!pip install pyfolio --quiet

In [ ]:
import pandas as pd
import chardet
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import dtw, dtw_path
from tslearn.utils import to_time_series_dataset

import networkx as nx

import FinanceDataReader as fdr
from pykrx import stock

import gym
from gym import spaces
import optuna
from stable_baselines3 import DDPG
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

from pykrx import stock
import yfinance as yf

# import cv2
# from google.colab.patches import cv2_imshow
import warnings

import gym
from gym import spaces
import optuna
from stable_baselines3 import DDPG
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [ ]:
import matplotlib.pyplot as plt # 맷플롯립 import 하기
plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기
plt.rc("axes", unicode_minus = False)

In [ ]:
df_korea = pd.read_excel('data/한국2.xlsx')
df_usa = pd.read_excel('data/미국2.xlsx')

df_kospi = fdr.StockListing('KOSPI')
df_snp500 = fdr.StockListing('S&P500')
df_nasdaq = fdr.StockListing('NASDAQ')

In [ ]:
df_korea['Code'] = [i[1:] for i in df_korea['Code']]
df_korea.index = range(len(df_korea))

In [ ]:
# 한국 재무제표 종목 - KOSPI 상장 종목 매칭
kospi_list = list(df_kospi['Code'])
lst = []
for i in range(len(df_korea['Code'])):
  if df_korea.iloc[i, 0] not in kospi_list:
    lst.append(i)

df_korea.drop(lst, axis=0, inplace=True)

df_korea.head()

In [ ]:
df_korea.drop_duplicates(subset='Code', keep='first', inplace=True) # 중복 제거
df_korea

In [ ]:
len(df_korea['Code'].unique())
df_usa.head()

In [ ]:
df_usa['Code'] = [i[:-3] for i in df_usa['Code']]
df_usa['Code'] = df_usa['Code'].str.strip()
df_usa.index = range(len(df_usa))
df_usa

In [ ]:
# 미국 재무제표 종목 - SnP500, NASDAQ 상장 종목 매칭
df_snp500['Symbol'] = df_snp500['Symbol'].str.strip()
df_nasdaq['Symbol'] = df_nasdaq['Symbol'].str.strip()

usa_list = list(df_usa.index)
snp_list = list(df_snp500.index)
nasdaq_list = list(df_nasdaq.index)

fdr_list = snp_list + nasdaq_list
fdr_list = set(fdr_list)

lst = []
for i in usa_list:
    if i in usa_list:
        lst.append(i)

len(lst)

In [ ]:
df_usa = df_usa.loc[lst,:]
df_usa

In [ ]:
korea_columns = set(df_korea.columns)
usa_columns = set(df_usa.columns)

if korea_columns == usa_columns:
    print("두 데이터프레임의 열이 동일합니다.")
else:
    print("두 데이터프레임의 열이 다릅니다.")

    # 어떤 열이 다른지 확인
    diff_columns_korea = korea_columns - usa_columns
    diff_columns_usa = usa_columns - korea_columns

    print("한국 데이터프레임에만 있는 열:", diff_columns_korea) # 비유동부채비율, EBITDA/순이자비용
    print("미국 데이터프레임에만 있는 열:", diff_columns_usa) # EBITDA/이자비용

In [ ]:
# 미국-한국 union
columns_to_drop_korea = [col for col in df_korea.columns if '비유동부채비율' in col or 'EBITDA/순이자비용' in col]
df_korea = df_korea.drop(columns=columns_to_drop_korea)

columns_to_drop_usa = [col for col in df_usa.columns if 'EBITDA/이자비용' in col]
df_usa = df_usa.drop(columns=columns_to_drop_usa)

# 두 데이터프레임을 union (concatenate) 합치기
result_df = pd.concat([df_korea, df_usa], ignore_index=True)

result_df.shape

In [ ]:
def create_melted_df(df, keyword):
    filtered = ['Code', 'Name'] + [col for col in df.columns if keyword in col]
    df1 = df[filtered]

    # 컬럼명을 날짜 형태로 변경
    selected_cols = df1.columns[df1.columns.str.contains(keyword)]
    df1 = df1.copy()

    start_date = pd.to_datetime('2010-01-01')
    date_columns = [start_date + pd.DateOffset(months=3 * i) for i in range(len(selected_cols))]
    date_column_mapping = dict(zip(selected_cols, date_columns))

    df1.rename(columns=date_column_mapping, inplace=True)

    date_column_mapping = {col: col.strftime('%Y-%m-%d') if col in date_columns else col for col in df1.columns}
    df1.rename(columns=date_column_mapping, inplace=True)

    # 'Code'와 'Name'을 기준으로 데이터를 재구성
    melted_df = pd.melt(df1, id_vars=['Code', 'Name'], var_name='시점', value_name=keyword)

    # '시점' 컬럼을 날짜 형식으로 변환
    melted_df['시점'] = pd.to_datetime(melted_df['시점'], errors='coerce')

    return melted_df

In [ ]:
# 키워드 리스트 정의 (26개)
keywords = ['영업이익률', '세전계속사업이익률', '순이익률', '총포괄이익률', 'EBITDA마진율',
            'ROE', 'ROA', 'ROIC', '매출원가율', '총자산영업이익률',
            '자기자본영업이익률', '자본금영업이익률', '부채비율', '유동비율', '당좌비율',
            '유동부채비율', '자기자본비율', '이자보상배율', '순이자보상배율', '비유동비율',
            '차입금비율', '비유동적합률', '총자산회전율', '매출채권회전율', '재고자산회전율',
            '매입채무회전율']

# 키워드별로 데이터프레임 생성
melted_dfs = [create_melted_df(result_df, keyword) for keyword in keywords]

In [ ]:
# 행 수 147,774
df1_len = len(df1) // 4   # 영업이익률
df13_len = len(df13) // 2 # 부채비율
df14_len = len(df14) // 2 # 유동비율
df18_len = len(df18) // 2 # 이자보상배율

df1 = df1.iloc[:df1_len]
df13 = df13.iloc[:df13_len]
df14 = df14.iloc[:df14_len]
df18 = df18.iloc[:df18_len]

print(len(df1), ' ', len(df13), ' ', len(df14), ' ', len(df18))

In [ ]:
# 6개의 데이터프레임을 melted_dfs 리스트에 저장한 후 merge
melted_dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,
              df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,
              df21, df22, df23, df24, df25, df26]

# 초기 조인 대상 데이터프레임을 melted_dfs[0]으로 설정
merged_df = melted_dfs[0]

# 나머지 데이터프레임을 순회하면서 조인
for df in melted_dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['Code', 'Name', '시점'], how='inner')

# 최종 데이터프레임 확인
merged_df

In [ ]:
result = merged_df.copy()

# 날짜형식 변환
result['시점'] = pd.to_datetime(result['시점'], format='%Y-%m-%d')
result['시점']

In [ ]:
# 최초 nan 값 발생 시점 이전은 0으로 대체
# Term을 기준으로 그룹화하고, 각 그룹에서 처음 nan 값이 시작된 시점을 파악하여 그 시점 이전의 nan 값을 0으로 대체
result['시점'] = result.groupby('Code')['시점'].transform(lambda x: x.ffill().bfill())
result = result.fillna(0)

result

In [ ]:
result.to_csv('한국미국종목별_재무제표26개총합.csv', index=False)

In [ ]:
df = pd.read_excel('data/한국미국종목별_재무제표26개총합2.xlsx')
df

In [ ]:
# 블룸버그 공급망 데이터
bloom = pd.read_csv('./BigConDataset/NHIS_BDC_BLOOMBERG_SLY_CPE_LNK')
bloom.columns = ['회사코드', 'FIGI코드', '회사공식명',
                 '관계회사코드', '관계회사FIGI코드', '관계회사공식명',
                '관계형태', '관계회계연도', '관계기간금액', '통화', '관계원천',
                '관계시작일', '관계종료일', '수신일자']
bloom.head()

In [ ]:
# 공급을 많이 하는 회사 = CUSTOMER를 많이 보유한 회사
bloom2 = bloom[bloom['관계형태']=='CUSTOMER'] # supplier.
bloom2
# 회사가: UniTest, 관계회사:한국전력. 관계형태: Customer
# 한국전력이 UniTest의 고객이다~



In [ ]:
# 최근순으로 정렬
bloom2.sort_values('관계회계연도', ascending=False)
# 중복 제거
bloom2 = bloom2.drop_duplicates(subset=['FIGI코드', '관계회사FIGI코드'], keep='first').reset_index(drop=True)
bloom2

In [ ]:
bloom2.to_csv('data/블룸버그_고객관계데이터.csv', index=False)

In [ ]:
bloom2.to_csv('data/블룸버그_고객관계데이터.csv', index=False)

In [ ]:
df['FIGI Code'] = df['FIGI Code'].str.replace(' ', '')  # 'FIGI Code' 컬럼의 공백 제거
bloom['FIGI코드'] = bloom['FIGI코드'].str.replace(' ', '')  # 'FIGI코드' 컬럼의 공백 제거
bloom['관계회사FIGI코드'] = bloom['관계회사FIGI코드'].str.replace(' ', '')

In [ ]:
df['시점'] = pd.to_datetime(df['시점'])
df2 = df.groupby('FIGI Code').apply(lambda x: x.loc[x['시점'].idxmax()]).reset_index(drop=True)
df2

In [ ]:
G = nx.from_pandas_edgelist(bloom, 'FIGI코드', '관계회사FIGI코드', create_using=nx.DiGraph())
centrality = nx.degree_centrality(G) # 관계 기반으로 가중치 부여

In [ ]:
plt.figure(figsize=(20,10))

# 중심성이 높은 상위 50개 노드 선택
top_50_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:50]
top_50_nodes = [node for node, centrality in top_50_nodes]

# 상위 50개 노드로 구성된 서브그래프 생성
subgraph = G.subgraph(top_50_nodes)

# 서브그래프 시각화
nx.draw(subgraph, with_labels=True)
plt.show()

# K-means K개의 중심점 -> 중심성이 높은(고객이 많은) 회사를 중심으로
# 고객이 많은 회사끼리 묶인다??(X)
# 재무제표를 기반으로 클러스터링.
# 기존: 재무제표가지고 전체 재무제표의 패턴을 기반으로하는거였다면
# 현재: 중심성 높은 특정 A회사가 기준점이되는.(아마존과 재무제표비슷한클러스터.., 삼성과 비슷한클러스터..)
# 가정: A회사와 공급-고객 관계를 맺고 있는 B,C회사들이 A회사와 재무제표 흐름이 비슷하다면.

In [ ]:
df['weight'] = df['FIGI Code'].map(centrality)
df

In [ ]:
df2 = df.sort_values('weight', ascending=False)
df2 = df2.drop_duplicates(subset=['FIGI Code'], keep='first').reset_index(drop=True)
df2.head(50)

In [ ]:
df['weight'] = df['weight'].fillna(0)
df.isna().sum()

In [ ]:
group = df.groupby(['Code', 'FIGI Code', 'Name']).mean()
group.head()

In [ ]:
X = group.iloc[:, 0:-1].values  # 3번째 열 이후의 열이 시계열 데이터

# 데이터 전처리: 시계열 데이터 스케일링
X = TimeSeriesScalerMeanVariance().fit_transform(X)

In [ ]:
# 3D 배열을 2D 배열로 변환
n_samples, n_timestamps, n_features = X.shape
X_2d = X.reshape(n_samples, -1)

# 시도할 클러스터 개수 범위
min_clusters = 2
max_clusters = 10

best_silhouette_score = -1
best_n_clusters = 0

for n_clusters in range(min_clusters, max_clusters + 1):
    model = TimeSeriesKMeans(n_clusters=n_clusters, verbose=False, random_state=0)
    y_pred = model.fit_predict(X)

    # 실루엣 점수 계산
    silhouette_avg = silhouette_score(X_2d, y_pred)

    # 가장 높은 실루엣 점수를 갖는 클러스터 개수 찾기
    if silhouette_avg > best_silhouette_score:
        best_silhouette_score = silhouette_avg
        best_n_clusters = n_clusters

print(f"Best number of clusters: {best_n_clusters}")
print(f"Best silhouette score: {best_silhouette_score}")

In [ ]:
class WeightedDTW(TimeSeriesKMeans):
    def __init__(self, weights, *args, **kwargs):
        self.weights = weights
        super().__init__(*args, **kwargs)

    def _transform_to_3D(self, X):
        X_3D = to_time_series_dataset(X)
        weights_3D = to_time_series_dataset(self.weights)
        return X_3D * weights_3D

    def _dtw(self, X, X_bis, *args, **kwargs):
        X_3D = self._transform_to_3D(X)
        X_bis_3D = self._transform_to_3D(X_bis)
        return dtw(X_3D, X_bis_3D, *args, **kwargs)

    def _dtw_path(self, X, X_bis, *args, **kwargs):
        X_3D = self._transform_to_3D(X)
        X_bis_3D = self._transform_to_3D(X_bis)
        return dtw_path(X_3D, X_bis_3D, *args, **kwargs)

In [ ]:
n_clusters = best_n_clusters
weights = group['weight'].values

model = WeightedDTW(weights=weights, n_clusters=n_clusters, verbose=True, random_state=0)
y_pred = model.fit_predict(X)

In [ ]:
# 클러스터링 결과 출력
for cluster in range(n_clusters):
    cluster_samples = np.where(y_pred == cluster)[0]
    print(f"Cluster {cluster}: {len(cluster_samples)}")

In [ ]:
group['Cluster'] = y_pred + 1 # 0,1,2 -> 1,2,3
group.reset_index(drop=False, inplace=True)
group.head()

In [ ]:
df = df.merge(group[['FIGI Code','Cluster']], on='FIGI Code', how='left')
df

In [ ]:
plt.figure(figsize=(10,18))
# 클러스터별 특징 파악
temp = df.groupby('Cluster').mean(numeric_only=True)
cluster_mean = temp.transpose()
mean_table = cluster_mean.div(cluster_mean.max(axis=1), axis=0)

plt.figure(figsize=(10,8))
annot_kws = {'fontsize':12}
vmin, vmax = 0 ,1
sns.heatmap(mean_table, annot=True, fmt='.2f',
           linewidths=0.1, annot_kws=annot_kws,
           cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
plt.show()

In [ ]:
num_cols = ['영업이익률', '세전계속사업이익률', '순이익률',
       '총포괄이익률', 'EBITDA마진율', 'ROE', 'ROA', 'ROIC', '매출원가율', '총자산영업이익률',
       '자기자본영업이익률', '자본금영업이익률', '부채비율', '유동비율', '당좌비율', '유동부채비율', '자기자본비율',
       '이자보상배율', '순이자보상배율', '비유동비율', '차입금비율', '비유동적합률', '총자산회전율', '매출채권회전율',
       '재고자산회전율', '매입채무회전율']

def plot_clustered_line_chart(data, y_column):
    # 'Cluster'를 기준으로 그룹화하고 y_column 평균을 계산
    grouped_data = data.groupby(['Cluster', '시점'])[y_column].mean().reset_index()

    # 각 클러스터에 대한 라인 차트 시각화
    plt.figure(figsize=(15, 6))

    for cluster in grouped_data['Cluster'].unique():
        cluster_data = grouped_data[grouped_data['Cluster'] == cluster]
        plt.plot(cluster_data['시점'], cluster_data[y_column], '-o', label=f'Cluster {cluster}')

    plt.xlabel('시점')
    plt.ylabel(f'{y_column} 평균')
    plt.xticks(rotation=45)  # x축 레이블을 45도 회전
    plt.legend()
    plt.title(f'클러스터별 {y_column} 평균 라인 차트')
    plt.grid(axis='y', linestyle='--')  # 가로 grid 줄 추가

    plt.show()

# 함수 호출
for y_column in num_cols:
    plot_clustered_line_chart(df, y_column)

In [ ]:
# PCA 주성분분석, 차원축소
pca = PCA(n_components=15)
pca.fit(df[num_cols])
df_pca = pca.transform(group[num_cols])

print('원본 데이터 형태: ', str(df.shape))
print('축소된 데이터 형태:', str(df_pca.shape))

In [ ]:
print('PCA 주성분 형태: ', pca.components_.shape)
print('PCA 주성분: ', pca.components_)

In [ ]:
# 각 주성분의 원본 데이터에 대한 설명력 확인
print('각 주성분의 분산 비율: ', pca.explained_variance_ratio_)

In [ ]:
num_components = range(1, min(len(group[num_cols].columns), len(group[num_cols])))
cumulative_variance = []
for n in num_components:
    pca = PCA(n_components=n)
    pca.fit(group[num_cols])
    cumulative_variance.append(np.sum(pca.explained_variance_ratio_))

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(num_components, cumulative_variance, marker='o', linestyle='-')
plt.title('주성분 개수에 따른 누적 설명 분산')
plt.xlabel('주성분 개수')
plt.ylabel('누적 설명 분산')
plt.grid(True)
plt.show()
# 주성분 2개일때 데이터셋 99% 이상 설명

In [ ]:
plt.figure(figsize=(15,13))
plt.matshow(pca.components_, cmap='RdYlBu_r', fignum=1)

plt.yticks([0,2])
plt.colorbar()

plt.xticks(range(len(group[num_cols].columns)), group[num_cols].columns, rotation=80, ha='left')

plt.xlabel('특성')
plt.ylabel('주성분')

for i in range(len(pca.components_)):
    for j in range(len(group[num_cols].columns)):
        text = f'{pca.components_[i, j]:.2f}'
        plt.text(j, i, text, va='center', ha='center', color='black')

## 주성분별 상관성 0.4 이상 변수
# 주성분1: 자본금영업이익률
# 주성분2: 영업이익률, 세전계속사업이익률, 순이익률, 총포괄이익률, EBITDA마진율

In [ ]:
# 2 > 1> 4 > 7 > 3 > 6 > 5
df.groupby('Cluster')['weight'].mean()

In [ ]:
data = pd.read_csv('data/종목별클러스터(블룸버그반영).csv')
data

In [ ]:
# 'weight'를 기준으로 내림차순 정렬하여 상위 10개 기업을 선택하는 함수
def get_top_10_by_weight(group):
    return group.nlargest(10, 'weight')

# 'Cluster'별로 상위 10개 기업을 선택하여 출력
top_10_by_cluster = data.groupby('Cluster').apply(get_top_10_by_weight)
top_10_by_cluster[:10]

In [ ]:
# 고객 데이터
cust = pd.read_csv('./BigConDataset/NHIS_BDC_NHDATA_CUS_TP_HLD_IEM')
cust.columns = ['기준일자', '종목코드', '고객구성대분류코드', '고객구성중분류코드', '보유종목분류기준코드', '보유기준종목코드', '보유종목순위기준']
cust

In [ ]:
# 한국/미국 종목만 남기기
# 각 조건을 별도의 변수에 저장
cond1 = cust['종목코드'].str.startswith('A')
cond2 = cust['종목코드'].str.startswith('US')
cond3 = cust['보유기준종목코드'].str.startswith('A')
cond4 = cust['보유기준종목코드'].str.startswith('US')

# 조건 결합
final_condition = (cond1 | cond2) & (cond3 | cond4)

In [ ]:
# 최종 조건에 해당하지 않는 행의 인덱스를 찾아 제거
customer = cust[final_condition].copy()

customer

In [ ]:
customer.to_csv('data/개인투자자보유종목(한미종목).csv', index=False)

In [ ]:
customer = pd.read_csv('data/개인투자자보유종목(한미종목).csv')
customer.head()

In [ ]:
# 연령대 대분류 고객만 선택
cat_cust = customer[customer['고객구성대분류코드']==2]
cat_cust

In [ ]:
# 고객 데이터 ISIN코드 미국(US)만 필터링
us_cust = cat_cust[cat_cust['종목코드'].str.contains('US')]
us_cust

In [ ]:
# 고객데이터 ISIN코드 중복 제거, 리스트에 담기
isin_lst = us_cust['종목코드'].unique().tolist()
len(isin_lst)

In [ ]:
temp = us_cust[us_cust['보유기준종목코드'].str.contains('US')]
temp

In [ ]:
temp_lst = temp['보유기준종목코드'].unique().tolist()
len(temp_lst)

In [ ]:
# 최종 종목 리스트 생성
final_lst = list(set(isin_lst + temp_lst))
len(final_lst)

In [ ]:
# 해당 리스트 내 ISIN 코드 검색 -> TICKER명 크롤링
def fetch_ticker_values(isin):
    # The URL where the data is to be fetched from
    url = f"https://www.morningstar.com/search?query={isin}"

    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all span elements with class 'mdc-security-module__ticker'
        ticker_elements = soup.find_all("span", class_="mdc-security-module__ticker")

        # Extracting text (ticker values) from each span element
        tickers = [element.get_text(strip=True) for element in ticker_elements]

        return tickers[0] if tickers else None
    else:
        return None

In [ ]:
ticker_dict = {}
for isin in tqdm(final_lst, desc="Fetching ticker values"):
    ticker = fetch_ticker_values(isin)
    if ticker is not None:
        ticker_dict[isin] = ticker

In [ ]:
# 티커코드 목록 저장
with open('tickers.txt', 'w') as f:
    for isin, ticker in ticker_dict.items():
        f.write(f"{isin}: {ticker}\n")

In [ ]:
ticker_dict = {}

with open('tickers.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line:
            ticker, symbol = line.split(':')
            ticker_dict[ticker.strip()] = symbol.strip()

In [ ]:
# '종목코드' 컬럼의 ISIN코드를 해당하는 티커코드로 수정
us_cust['종목코드'] = us_cust['종목코드'].map(lambda x: ticker_dict.get(x, x))
us_cust['보유기준종목코드'] = us_cust['보유기준종목코드'].map(lambda x: ticker_dict.get(x, x))

us_cust

In [ ]:
kr_cust = cat_cust[cat_cust['종목코드'].str.contains('A')]
kr_cust['보유기준종목코드'] = kr_cust['보유기준종목코드'].map(lambda x: ticker_dict.get(x, x))

kr_cust

In [ ]:
result_cust = pd.concat([us_cust, kr_cust], axis=0).drop_duplicates()
## 고객 데이터 완성

In [ ]:
result_cust = pd.concat([us_cust, kr_cust], axis=0).drop_duplicates()
## 고객 데이터 완성

In [ ]:
result_cust.to_csv('data/연령대별고객보유종목데이터.csv', index=False)

In [ ]:
result_cust = pd.read_csv('data/연령대별고객보유종목데이터.csv')
result_cust

In [ ]:
# 고객데이터와 재무제표 데이터 결합 (한국은 "A" + 재무제표데이터 Code / 미국은 그대로)
# 얘는 재무제표 데이터 + 클러스터 매핑된 데이터

df = pd.read_excel('data/종목별클러스터(블룸버그반영).xlsx')
df_temp = df.copy()
df

In [ ]:
df = df.sort_values('Code')
kr_df = df[:46037]
kr_df['Code'] = kr_df['Code'].apply(lambda x: 'A' + x)
us_df = df[46037:]
result_df = pd.concat([kr_df, us_df], axis=0)
## 재무제표 데이터 완성

In [ ]:
result_df.to_csv('data/종목별클러스터(블룸버그반영).csv', index=False)
result_df

In [ ]:
result_df = pd.read_csv('data/종목별클러스터(블룸버그반영).csv')
result_df


In [ ]:
result_cust = pd.read_csv('data/연령대별고객보유종목데이터.csv')
result_cust

In [ ]:
# 연령대에 따른 고객 구분
cust1 = result_cust[result_cust['고객구성중분류코드'] == 21] # 0~29세
cust2 = result_cust[result_cust['고객구성중분류코드'] == 22] # 30~39세
cust3 = result_cust[result_cust['고객구성중분류코드'] == 23] # 40~49세
cust4 = result_cust[result_cust['고객구성중분류코드'] == 24] # 50~59세
cust5 = result_cust[result_cust['고객구성중분류코드'] == 25] # 60세~

In [ ]:
result_cust = result_cust.merge(result_df[['Code', 'Cluster']], left_on='종목코드', right_on='Code')
result_cust = result_cust.drop(columns='Code')
result_cust = result_cust.rename(columns={'Cluster': '종목코드 Cluster'})

result_cust = result_cust.merge(result_df[['Code', 'Cluster']], left_on='보유기준종목코드', right_on='Code')
result_cust = result_cust.drop(columns='Code')
result_cust = result_cust.rename(columns={'Cluster': '보유기준종목코드 Cluster'})

In [ ]:
result_cust = result_cust.merge(result_df[['Code', 'Cluster']], left_on='종목코드', right_on='Code')
result_cust = result_cust.drop(columns='Code')
result_cust = result_cust.rename(columns={'Cluster': '종목코드 Cluster'})

result_cust = result_cust.merge(result_df[['Code', 'Cluster']], left_on='보유기준종목코드', right_on='Code')
result_cust = result_cust.drop(columns='Code')
result_cust = result_cust.rename(columns={'Cluster': '보유기준종목코드 Cluster'})

In [ ]:
# 중복 제거 후 '종목코드 Cluster-보유기준종목코드 Cluster' 쌍의 count 계산
counts = cust2.drop_duplicates().groupby(['종목코드 Cluster', '보유기준종목코드 Cluster']).size().sort_values(ascending=True)

# 막대 그래프 시각화
counts.plot(kind='barh')
plt.xlabel('종목코드 Cluster - 보유기준종목코드 Cluster')
plt.ylabel('Count')
plt.title('고객 연령대 30~39세')
plt.show()

In [ ]:
# 중복 제거 후 '종목코드 Cluster-보유기준종목코드 Cluster' 쌍의 count 계산
counts = cust3.drop_duplicates().groupby(['종목코드 Cluster', '보유기준종목코드 Cluster']).size().sort_values(ascending=True)

# 막대 그래프 시각화
counts.plot(kind='barh')
plt.xlabel('종목코드 Cluster - 보유기준종목코드 Cluster')
plt.ylabel('Count')
plt.title('고객 연령대 40~49세')
plt.show()

In [ ]:
# 중복 제거 후 '종목코드 Cluster-보유기준종목코드 Cluster' 쌍의 count 계산
counts = cust4.drop_duplicates().groupby(['종목코드 Cluster', '보유기준종목코드 Cluster']).size().sort_values(ascending=True)

# 막대 그래프 시각화
counts.plot(kind='barh')
plt.xlabel('종목코드 Cluster - 보유기준종목코드 Cluster')
plt.ylabel('Count')
plt.title('고객 연령대 50~59세')
plt.show()

In [ ]:
# 중복 제거 후 '종목코드 Cluster-보유기준종목코드 Cluster' 쌍의 count 계산
counts = cust5.drop_duplicates().groupby(['종목코드 Cluster', '보유기준종목코드 Cluster']).size().sort_values(ascending=True)

# 막대 그래프 시각화
counts.plot(kind='barh')
plt.xlabel('종목코드 Cluster - 보유기준종목코드 Cluster')
plt.ylabel('Count')
plt.title('고객 연령대 60세 이상')
plt.show()

In [ ]:
# cust2 필터링
# '보유종목순위기준'이 1, 2, 3에 해당하는 행 추출
cust2 = cust2[cust2['보유종목순위기준'].isin([1, 2, 3])]

# '종목코드'와 '보유기준종목코드'가 모두 중복되는 행 제거
cust2 = cust2.drop_duplicates(subset=['종목코드', '보유기준종목코드'])

In [ ]:
cust2 = cust2.merge(result_df[['Code', 'Cluster']], left_on='종목코드', right_on='Code')
cust2 = cust2.drop(columns='Code')
cust2 = cust2.rename(columns={'Cluster': '종목코드 Cluster'})

cust2 = cust2.merge(result_df[['Code', 'Cluster']], left_on='보유기준종목코드', right_on='Code')
cust2 = cust2.drop(columns='Code')
cust2 = cust2.rename(columns={'Cluster': '보유기준종목코드 Cluster'})


In [ ]:
# ~29세 고객이 보유한 종목만 필터링
# '종목코드'와 '보유기준종목코드' unique 값 추출
unique_codes = cust2['종목코드'].unique().tolist() + cust2['보유기준종목코드'].unique().tolist()
unique_codes = list(set(unique_codes))
unique_codes = [item.strip() for item in unique_codes]
unique_codes[:5]


In [ ]:
# 'df'에서 'Code'가 unique_codes에 포함되는 행만 남기기
cust2_df = result_df[result_df['Code'].isin(unique_codes)]
cust2_df

In [ ]:
cust2_df_cluster5 = cust2_df[cust2_df['Cluster']==5]
cust2_df_cluster2 = cust2_df[cust2_df['Cluster']==2]
cust2_df_cluster1 = cust2_df[cust2_df['Cluster']==1]

In [ ]:
cust2_df_cluster5 = cust2_df_cluster5.iloc[:, 1:].reset_index(drop=True)
cust2_df_cluster2 = cust2_df_cluster2.iloc[:, 1:].reset_index(drop=True)
cust2_df_cluster1 = cust2_df_cluster1.iloc[:, 1:].reset_index(drop=True)

In [ ]:
cust2_df_cluster5.to_csv('data/30-39세_고객_클러스터5번_보유종목.csv', index=False)
cust2_df_cluster2.to_csv('data/30-39세_고객_클러스터2번_보유종목.csv', index=False)
cust2_df_cluster1.to_csv('data/30-39세_고객_클러스터1번_보유종목.csv', index=False)

In [ ]:
## 3. DDPG 강화학습

- 고객군마다 해당되는 대표 클러스터별로 DDPG 강화학습 모델 구축
- 강화학습 모델은 종목 등락률로 학습, 샤프지수로 평가. 결과값으로 종목별 투자 시그널 제시, 효율적 투자선/샤프지수 포트폴리오 시각화
- 몬테카를로 시뮬레이션 추가하여 강화학습 모델 개선을 위한 파라미터 설정, 시장의 변화에 대응할 수 있는 인사이트를 도출
- 고객 구분(자산규모)에 따른 클러스터별 투자 시그널 on인 종목들 결과 종합, 종목의 특성을 확인하여, 고객들의 권장 투자 패턴 도출.

In [ ]:
import pandas as pd
df = pd.read_csv('data/30-39세_고객_클러스터5번_보유종목.csv')
df

In [ ]:
def split_korea_us(tickers):
    korea = []
    usa = []

    # 티커를 검사하여 적절한 리스트에 추가
    for ticker in tickers:
        if ticker[1:].isdigit():
            korea.append(ticker)
        else:
            usa.append(ticker)
    print("korea:", len(korea))
    print("usa:", len(usa))

    return korea, usa

In [ ]:
tickers = list(df['Code'].unique())
korea, usa = split_korea_us(tickers)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 현재 날짜와 시간을 얻음
current_date = datetime.now()

# 3년 전 날짜 계산 <-  일관되게 수익을 내고 위험을 관리하는 종목 선정 위해
years_ago = current_date - relativedelta(years=3)

# 형식에 맞게 날짜를 문자열로 변환
start = years_ago.strftime('%Y-%m-%d')

# 현재 날짜를 문자열로 변환
end = current_date.strftime('%Y-%m-%d')


In [ ]:
def us_sharpe_ratio(usa, start):
    data = yf.download(usa, start=start)
    data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
    data = data.droplevel(0,axis=1)
    rets = data.pct_change().fillna(0)
    sharpe_ratio = (rets.mean() * np.sqrt(252)) / rets.std()

    return sharpe_ratio

sharpe_ratio = us_sharpe_ratio(usa, start)

In [ ]:
def korea_sharpe_ratio(korea, start, end):
    # 각 종목별 데이터를 저장할 빈 딕셔너리 생성
    data_dict = {}

    # 각 종목별로 데이터를 가져와 딕셔너리에 저장
    for ticker in korea:
        data_dict[ticker[1:]] = stock.get_market_ohlcv_by_date(start,end,ticker[1:])
    results = {}
    for ticker, df in data_dict.items():
        results[ticker] = data_dict[ticker]['종가']
    df = pd.DataFrame(results)

    rets = df.pct_change().fillna(0)
    sharpe_ratio2 = (rets.mean() * np.sqrt(252)) / rets.std()

    return sharpe_ratio2

sharpe_ratio2 = korea_sharpe_ratio(korea, start, end)

In [ ]:
df_total = pd.concat([sharpe_ratio, sharpe_ratio2])
df_total = df_total.sort_values(ascending=False)
df_total = df_total.sort_values(ascending=False)
df_cut = df_total[df_total>=0.5]

In [ ]:
tickers = df_cut.index.unique()
korea, usa = split_korea_us(tickers)

In [ ]:
### from datetime import datetime

# 현재 날짜와 시간을 얻음
current_date = datetime.now()

end= current_date.strftime('%Y-%m-%d')

from datetime import timedelta

# 사용자 기간 설정
days = int(input())

days_ago = current_date - timedelta(days=days)

# 형식에 맞게 날짜를 문자열로 변환
start = days_ago.strftime('%Y-%m-%d')


In [ ]:
def us_fluctuation_rate(usa, start):
    data = yf.download(usa, start=start)
    data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
    data = data.droplevel(0,axis=1)
    daterange = pd.date_range(start=start, end=end)
    df_stocks = pd.DataFrame(index=daterange)
    df = df_stocks.join(data).fillna(method='ffill')
    rets = df.pct_change().fillna(0)
    df = rets

    df_stocks = pd.DataFrame()
    cols = list(df.columns)

    for i in cols:
        a = pd.DataFrame(df[i].values, index = df.index, columns = ['등락률'])
        b = pd.DataFrame([i]*len(df.index), index = df.index, columns=['종목'])
        c = pd.concat([a,b], axis=1)
        df_stocks = pd.concat([df_stocks, c], axis = 0)

    return df_stocks

df_stocks = us_fluctuation_rate(usa, start)

In [ ]:
def korea_fluctuation_rate(korea, start, end):
    # 각 종목별 데이터를 저장할 빈 딕셔너리 생성
    data_dict = {}

    # 각 종목별로 데이터를 가져와 딕셔너리에 저장
    for ticker in korea:
        data_dict[ticker] = stock.get_market_ohlcv_by_date(start,end,ticker)
    results = {}
    for ticker, df in data_dict.items():
        results[ticker] = data_dict[ticker]['종가']
    df = pd.DataFrame(results)
    daterange = pd.date_range(start=start, end=end)
    df_stocks2 = pd.DataFrame(index=daterange)
    df = df_stocks2.join(df).fillna(method='ffill')
    rets = df.pct_change().fillna(0)
    df = rets
    df_stocks2 = pd.DataFrame()
    cols = list(df.columns)

    for i in cols:
        a = pd.DataFrame(df[i].values, index = df.index, columns = ['등락률'])
        b = pd.DataFrame([i]*len(df.index), index = df.index, columns=['종목'])
        c = pd.concat([a,b], axis=1)
        df_stocks2 = pd.concat([df_stocks2, c], axis = 0)

    return df_stocks2

df_stocks2 = korea_fluctuation_rate(korea, start, end)

In [ ]:
df_total = pd.concat([df_stocks, df_stocks2], axis=0)
df_total.rename(columns={'종목':'티커'}, inplace=True)
df = df_total
df = df.reset_index(drop=False)
df.rename(columns={'index': '날짜'}, inplace=True)


In [ ]:
# train, test셋을 나누기 위해 종목 기준으로 8:2로 나눈다.

def split_data_per_ticker(df, train_ratio=0.8):
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for ticker in df['티커'].unique():
        ticker_data = df[df['티커'] == ticker]

        split_index = int(len(ticker_data) * train_ratio)
        train_data = train_data.append(ticker_data[:split_index])
        test_data = test_data.append(ticker_data[split_index:])

    return train_data, test_data

# 데이터를 훈련 및 테스트 세트로 분할
df_train, df_test = split_data_per_ticker(df)

# 결과 확인
print("Train set size:", len(df_train))
print("Test set size:", len(df_test))
print(len(df_train['티커'].unique()))
print(len(df_test['티커'].unique()))


In [ ]:
import gym
import numpy as np
import pandas as pd
import optuna
from stable_baselines3 import DDPG
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv

# 1. StockTradingEnv: 주식 트레이딩 환경 정의
class StockTradingEnv(gym.Env):
    def __init__(self, df_train):
        super(StockTradingEnv, self).__init__()

        self.df = df_train
        self.current_step = 0
        self.portfolio_return_history = []

        # Action space: Proportions for each stock (Continuous action space)
        self.action_space = spaces.Box(low=0, high=1, shape=(len(df_train['티커'].unique()),))

        # State space: '등락률' for each stock
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(df_train['티커'].unique()),))

    def reset(self):
        self.current_step = 0
        #print("Resetting environment. Initial state:", self._next_observation())
        self.portfolio_return_history = []
        self.df.reset_index(drop=True, inplace=True)

        return self._next_observation()

    def _next_observation(self):

        obs = self.df.loc[self.current_step, '등락률']
        # If obs is not a Series (i.e., it's a scalar), make it into an array
        if not isinstance(obs, pd.Series):
            obs = np.array([obs])
        else:
            obs = obs.values
        return obs


    def step(self, action):
        # Ensure actions sum to 1 to represent allocation of capital between stocks
        action = action / np.sum(action)

        # Get the stock returns for the next day
        next_day_returns = self.df.loc[self.current_step, '등락률']

        # Calculate the portfolio return
        portfolio_return = np.dot(action, next_day_returns)
        self.portfolio_return_history.append(portfolio_return)

        # Calculate the reward using the Sharpe Ratio
        expected_return = np.mean(self.portfolio_return_history)
        risk_free_rate = 0.01  # for example, a 1% risk-free rate
        std_return = np.std(self.portfolio_return_history)
        if std_return == 0:  # to avoid division by zero
            sharpe_ratio = 0
        else:
            sharpe_ratio = (expected_return - risk_free_rate) / std_return

        reward = sharpe_ratio

        # Move to the next day
        self.current_step += 1

        # Get the new observation
        obs = self._next_observation()

        # Check if we're done
        done = self.current_step >= len(self.df) - 1

        return obs, reward, done, {}

# 주식 데이터(df)는 이전과 동일하게 정의합니다.

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch
import torch.nn as nn
from torch.nn import functional as F

class CustomNetwork(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=64):
        super(CustomNetwork, self).__init__(observation_space, features_dim)
        self.net = nn.Sequential(
            nn.Linear(observation_space.shape[0], 64),
            nn.ReLU(),
            nn.Dropout(0.5),  # 50% 드롭아웃
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, observations):
        return self.net(observations)

# 정책 네트워크의 구조 및 옵티마이저 설정
policy_kwargs = dict(
    features_extractor_class=CustomNetwork,
    features_extractor_kwargs=dict(features_dim=64),
    optimizer_class=torch.optim.Adam,
    optimizer_kwargs=dict(weight_decay=0.01)  # 가중치 감쇠 적용
)



# 2. objective: optuna의 목표함수 정의
def objective(trial):
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])

    env = DummyVecEnv([lambda: StockTradingEnv(df_train)])  # df는 주식 데이터의 데이터프레임입니다.
    model = DDPG('MlpPolicy', env, gamma=gamma, learning_rate=learning_rate, verbose=1, tensorboard_log="./tensorboard_logs",  policy_kwargs=policy_kwargs)
    model.learn(total_timesteps=20000)

    rewards = []
    obs = env.reset()
    for _ in range(1):  # 예: 1일 동안의 트레이딩을 시뮬레이션합니다.
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        rewards.append(reward)

    # 1일 동안의 보상의 평균을 반환합니다.
    avg_reward = np.mean(rewards)
    return avg_reward


# optuna를 사용하여 하이퍼파라미터 튜닝을 시작합니다.
study = optuna.create_study(direction="maximize")  # 보상을 최대화합니다.
study.optimize(objective, n_trials=3)  # 예: 30번의 시도로 최적의 하이퍼파라미터를 찾습니다.

# 최적의 하이퍼파라미터를 출력합니다.
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class PrintRewardCallback(BaseCallback):
    def __init__(self, check_freq: int, verbose=0):
        super(PrintRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.rewards = []

    def _on_step(self):
        # Check if ep_info_buffer is not empty
        if self.model.ep_info_buffer:
            self.rewards.append(self.model.ep_info_buffer[-1]['r'])  # Append the reward

        if self.n_calls % self.check_freq == 0 and self.rewards:
            mean_reward = np.mean(self.rewards[-self.check_freq:])
            print(f"Mean Reward: {mean_reward:.2f}")
        return True


In [ ]:
best_params = study.best_params

env = DummyVecEnv([lambda: StockTradingEnv(df_train)])
model = DDPG('MlpPolicy', env,
                gamma=best_params["gamma"],
                 learning_rate=best_params["learning_rate"],
                 batch_size=best_params["batch_size"],
                 verbose=1,
                 tensorboard_log="./tensorboard_logs",
                 policy_kwargs=policy_kwargs)

callback = PrintRewardCallback(check_freq=1000)
model.learn(total_timesteps=20000)

In [ ]:
obs = env.reset()
for _ in range(len(df_train)):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)

action

In [ ]:
import gym
import numpy as np
import pandas as pd
from gym import spaces

# 테스트 환경 정의 (기존 환경과 동일한 구조를 가집니다)
class StockTradingTestEnv(gym.Env):
    def __init__(self, df_test):
        super(StockTradingTestEnv, self).__init__()

        self.df = df_test
        self.current_step = 0
        self.portfolio_return_history = []

        # Action space: Proportions for each stock (Continuous action space)
        self.action_space = spaces.Box(low=0, high=1, shape=(len(df_test['티커'].unique()),))

        # State space: '등락률' for each stock
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(df_test['티커'].unique()),))

    def reset(self):
        self.current_step = 0
        #print("Resetting environment. Initial state:", self._next_observation())
        self.portfolio_return_history = []
        self.df.reset_index(drop=True, inplace=True)

        return self._next_observation()

    def _next_observation(self):

        obs = self.df.loc[self.current_step, '등락률']
        # If obs is not a Series (i.e., it's a scalar), make it into an array
        if not isinstance(obs, pd.Series):
            obs = np.array([obs])
        else:
            obs = obs.values
        return obs


    def step(self, action):
        # Ensure actions sum to 1 to represent allocation of capital between stocks
        action = action / np.sum(action)

        # Get the stock returns for the next day
        next_day_returns = self.df.loc[self.current_step, '등락률']

        # Calculate the portfolio return
        portfolio_return = np.dot(action, next_day_returns)
        self.portfolio_return_history.append(portfolio_return)

        # Calculate the reward using the Sharpe Ratio
        expected_return = np.mean(self.portfolio_return_history)
        risk_free_rate = 0.01  # for example, a 1% risk-free rate
        std_return = np.std(self.portfolio_return_history)
        if std_return == 0:  # to avoid division by zero
            sharpe_ratio = 0
        else:
            sharpe_ratio = (expected_return - risk_free_rate) / std_return

        reward = sharpe_ratio

        # Move to the next day
        self.current_step += 1

        # Get the new observation
        obs = self._next_observation()

        # Check if we're done
        done = self.current_step >= len(self.df) - 1

        return obs, reward, done, {}


def test_model(env, model, benchmark_returns):
    total_returns = []
    obs = env.reset()

    unique_dates = df_test['날짜'].unique()  # 날짜별로 유니크하게 추출

    for i in range(len(unique_dates)-1):
        next_date = unique_dates[i + 1]

        action, _ = model.predict(obs)
        obs, _, done, _ = env.step(action)

        # action을 Pandas Series로 변환
        action_series = pd.Series(action[0]/action[0].sum())

        # 다음 날짜에 해당하는 모든 종목의 등락률을 가져옵니다.
        next_day_returns = df_test[df_test['날짜'] == next_date]['등락률'].values
        print(next_day_returns)
        # 매수 비율과 다음 날의 등락률을 곱하여 일일 포트폴리오 수익률 계산
        daily_portfolio_return = np.dot(action_series, next_day_returns)

        # 수익률을 리스트에 추가합니다.
        total_returns.append(daily_portfolio_return)

    # 모델의 평균 일일 수익률을 계산합니다.
    model_mean_return = np.mean(total_returns)
    benchmark_mean_return = np.mean(benchmark_returns)

    print(f"Model Mean Return: {model_mean_return * 100}%")
    print(f"Benchmark Mean Return: {benchmark_mean_return * 100}%")

    success_rate = model_mean_return > benchmark_mean_return
    print(f"Model beats the benchmark: {success_rate}")

    return model_mean_return, benchmark_mean_return, success_rate



# 벤치마크: MSCI-> DM(선진시장) / EM(신흥시장) -> 미국 종목이 많을 경우 DM, 한국 종목이 많을 경우 EM

datetime_str = str(df_test['날짜'].unique()[0])

# 문자열에서 날짜 부분만 추출 ('T' 문자 전까지)
date_str = datetime_str.split('T')[0]

data = yf.download("URTH", start=date_str) # EEM
data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
rets = data.pct_change().fillna(0)

benchmark_returns = rets['Adj Close']  # 벤치마크 지수의 일별 수익률


# 테스트 환경 생성
test_env = DummyVecEnv([lambda: StockTradingTestEnv(df_test)])

# 모델 테스트
test_avg_reward, benchmark_avg_return, success_rate = test_model(test_env, model, benchmark_returns)


In [ ]:
model.save("model/ddpgV5")

In [ ]:
data_in_out = pd.read_csv('./BigConDataset/NHIS_BDC_NHDATA_IFW_OFW_RNK')
data_in_out.columns = ['유입유출구분코드', '기준일자', '종목코드', '유입유출기간코드', '유입유출종목코드', '유입유출순위']
data_in_out

In [ ]:
df_names = pd.read_csv('./data/종목별클러스터(블룸버그반영).csv')
df_names

In [ ]:
names_map = df_names.set_index('Code')['Name'].to_dict()

In [ ]:
df = data_in_out.copy()
df.head()

In [ ]:
file_path = './tickers.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

ticker_dict = {}
for line in lines:
    # Assuming each line is in the format "key: value"
    parts = line.strip().split(':')
    if len(parts) == 2:
        key, value = parts
        ticker_dict[key.strip()] = value.strip()


In [ ]:
file_path2 = './tickers2.txt'
with open(file_path2, 'r') as file:
    lines = file.readlines()

ticker_dict2 = {}
for line in lines:
    # Assuming each line is in the format "key: value"
    parts = line.strip().split(':')
    if len(parts) == 2:
        key, value = parts
        ticker_dict2[key.strip()] = value.strip()


In [ ]:
ticker_merged = dict(ticker_dict, **ticker_dict2)

In [ ]:
total_merged = dict(names_map, **ticker_merged)

In [ ]:
df['종목코드'] = df['종목코드'].str.strip()
df['종목코드'] = df['종목코드'].str.replace('A', '')
df['유입유출종목코드'] = df['유입유출종목코드'].str.strip()
df['유입유출종목코드'] = df['유입유출종목코드'].str.replace('A', '')

df['종목명'] = df['종목코드'].map(d4)
df['유입유출종목명'] = df['유입유출종목코드'].map(d4)

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
df_cluster5 = pd.read_csv('./data/30-39세_고객_클러스터5번_보유종목.csv')
df_cluster5

In [ ]:
clusters = df_cluster5.Name.unique()
inflow_outflow_data = df

In [ ]:
inflow_outflow_data

In [ ]:
cluster_inflow = inflow_outflow_data[(inflow_outflow_data['종목명'].isin(clusters)) & (inflow_outflow_data['유입유출구분코드'] == 1)]
cluster_outflow = inflow_outflow_data[(inflow_outflow_data['종목명'].isin(clusters)) & (inflow_outflow_data['유입유출구분코드'] == 2)]

In [ ]:
def adjust_portfolio_inflow(portfolio, inflow_data):
    for ticker in portfolio:
        if ticker in inflow_data['종목명'].values:
            inflow_rank = inflow_data[inflow_data['종목명'] == ticker]['유입유출순위'].iloc[0]
            portfolio[ticker]['weight'] += 1 / inflow_rank
    return portfolio


In [ ]:
def adjust_portfolio_outflow(portfolio, outflow_data, all_tickers):
    for ticker in portfolio:
        if ticker in outflow_data['종목명'].values:
            outflow_rank = outflow_data[outflow_data['종목명'] == ticker]['유입유출순위'].iloc[0]
            if outflow_rank < threshold:
                del portfolio[ticker]
            new_ticker = outflow_data[outflow_data['종목명'] == ticker]['유입유출종목명'].iloc[0]
            if new_ticker not in portfolio and new_ticker in all_tickers:
                portfolio[new_ticker] = {'weight': initial_weight}
    return portfolio


In [ ]:
portfolio_tickers = clusters

portfolio = {ticker: {'weight': 1/len(portfolio_tickers)} for ticker in portfolio_tickers}
all_tickers = set(inflow_outflow_data['종목명'].unique()) | set(inflow_outflow_data['유입유출종목명'].unique())

# 유입 데이터를 기반으로 포트폴리오 비중 조정
inflow_data = inflow_outflow_data[inflow_outflow_data['유입유출구분코드'] == '01']
outflow_data = inflow_outflow_data[inflow_outflow_data['유입유출구분코드'] == '02']

#portfolio = adjust_portfolio_inflow(portfolio, inflow_data)

In [ ]:
cluster_tickers = clusters

In [ ]:
reinforcement_portfolio_weights = action[0]

In [ ]:
reinforcement_portfolio_weights = [max(0, weight) for weight in reinforcement_portfolio_weights]

total_weight = sum(reinforcement_portfolio_weights)
normalized_weights = [weight / total_weight for weight in reinforcement_portfolio_weights]

sum(normalized_weights)


In [ ]:
selected_tickers = list(cluster_tickers)[:333]
len(selected_tickers)

In [ ]:
# 클러스터 종목 목록
cluster_tickers = selected_tickers

# 강화학습으로부터 받은 포트폴리오 가중치 (배열로 표현)
reinforcement_portfolio_weights = normalized_weights

# 종목과 인덱스 매핑
ticker_to_index = {ticker: index for index, ticker in enumerate(cluster_tickers)}

# 제거 및 추가된 종목을 추적하기 위한 리스트
removed_tickers = []
added_tickers = []

# 가중치 조정을 위한 팩터
adjustment_factor = 0.05  # 가중치 조정을 위한 팩터
outflow_threshold = 3     # 유출 데이터에서 종목을 제거하기 위한 순위 기준

# 유입 및 유출 데이터를 기반으로 가중치를 조정하는 함수
def adjust_weights_by_inflow_outflow(rl_weights, inflow_data, outflow_data):
    global removed_tickers, added_tickers

    # 유입 데이터에 따른 가중치 조정 및 종목 추가
    for _, row in inflow_data.iterrows():
        ticker = row['종목명']
        inflow_rank = row['유입유출순위']
        period_code = row['유입유출기간코드']
        # 기간 코드에 따른 가중치 조정 로직
        period_factor = 1 if period_code in [1, 2] else 2 if period_code == 3 else 3

        if ticker in ticker_to_index:
            index = ticker_to_index[ticker]
            rl_weights[index] += adjustment_factor * period_factor / inflow_rank

    # 유출 데이터에 따른 가중치 조정 및 종목 제거
    for _, row in outflow_data.iterrows():
        ticker = row['종목명']
        outflow_rank = row['유입유출순위']
        if ticker in ticker_to_index and outflow_rank < outflow_threshold:
            index = ticker_to_index[ticker]
            removed_tickers.append(ticker)
            rl_weights[index] = 0  # 가중치를 0으로 설정하여 종목 제거

    # 가중치 정규화 (합계가 1이 되도록)
    total_weight = sum(rl_weights)
    rl_weights = [weight / total_weight for weight in rl_weights]

    return rl_weights

# 유입 및 유출 데이터
# inflow_data, outflow_data - 이전 예제에서 정의된 DataFrame 사용

# 가중치 조정 실행
adjusted_portfolio_weights = adjust_weights_by_inflow_outflow(
    reinforcement_portfolio_weights,
    inflow_data,
    outflow_data
)

# 결과 출력
print("조정된 포트폴리오 가중치: ", adjusted_portfolio_weights)
print("제거된 종목: ", removed_tickers)
print("새로 편입된 종목: ", added_tickers)


In [ ]:
original_data = df_test
original_data

In [ ]:
original_data.reset_index(inplace=True)
original_data.rename(columns={'index':'date'}, inplace=True)
original_data.reset_index(inplace=True)


In [ ]:
pivoted_df = original_data.pivot(index='date', columns='티커', values='등락률')


In [ ]:
import pyfolio as pf
import pandas as pd
import numpy as np

# Assuming 'filled_pivoted_df' is your DataFrame and it has a date column
filled_pivoted_df = pivoted_df.apply(lambda col: col.fillna(col.mean()), axis=0)

# Convert the index of your DataFrame to DateTime if it's not already
if not isinstance(filled_pivoted_df.index, pd.DatetimeIndex):
    filled_pivoted_df.index = pd.to_datetime(filled_pivoted_df.index)

filled_pivoted_df = filled_pivoted_df.apply(lambda col: col.fillna(col.mean()), axis=0)
nan_info_after = filled_pivoted_df.isna().sum().sum()
print(filled_pivoted_df.head())
print("Remaining NaN values after filling:", nan_info_after)

# Calculate Portfolio Returns
n_assets = len(filled_pivoted_df.columns) # Assuming each column is an asset
portfolio_weights = n_assets * [1 / n_assets]
portfolio_returns = pd.Series(np.dot(portfolio_weights, filled_pivoted_df.T),
                              index=filled_pivoted_df.index)

# Create Pyfolio Tear Sheet
pf.create_simple_tear_sheet(portfolio_returns)


In [ ]:
N_PORTFOLIOS = 10 ** 5
N_DAYS = 252
RISKY_ASSETS = list(returns.columns.unique())
RISKY_ASSETS.sort()
START_DATE = str(returns.index[0]).split(' ')[0]
END_DATE = str(returns.index[-1]).split(' ')[0]

n_assets = len(RISKY_ASSETS)

In [ ]:
np.random.seed(42)
weights = np.random.random(size=(N_PORTFOLIOS, n_assets))
weights /=  np.sum(weights, axis=1)[:, np.newaxis]

In [ ]:
returns_df = filled_pivoted_df

In [ ]:
avg_returns = returns_df.mean() * N_DAYS
cov_mat = returns_df.cov() * N_DAYS

In [ ]:
portf_rtns = np.dot(weights, avg_returns)

portf_vol = []
for i in range(0, len(weights)):
    portf_vol.append(np.sqrt(np.dot(weights[i].T,
                                    np.dot(cov_mat, weights[i]))))
portf_vol = np.array(portf_vol)
portf_sharpe_ratio = portf_rtns / portf_vol

In [ ]:
portf_results_df = pd.DataFrame({'returns': portf_rtns,
                                 'volatility': portf_vol,
                                 'sharpe_ratio': portf_sharpe_ratio})

In [ ]:
N_POINTS = 100
portf_vol_ef = []
indices_to_skip = []

portf_rtns_ef = np.linspace(portf_results_df.returns.min(),
                            portf_results_df.returns.max(),
                            N_POINTS)
portf_rtns_ef = np.round(portf_rtns_ef, 2)
portf_rtns = np.round(portf_rtns, 2)

for point_index in range(N_POINTS):
    if portf_rtns_ef[point_index] not in portf_rtns:
        indices_to_skip.append(point_index)
        continue
    matched_ind = np.where(portf_rtns == portf_rtns_ef[point_index])
    portf_vol_ef.append(np.min(portf_vol[matched_ind]))

portf_rtns_ef = np.delete(portf_rtns_ef, indices_to_skip)

In [ ]:
#MARKS = ['o', 'X', 'd', '*']

fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter', x='volatility',
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black',
                      ax=ax)
ax.set(xlabel='Volatility',
       ylabel='Expected Returns',
       title='Efficient Frontier')
ax.plot(portf_vol_ef, portf_rtns_ef, 'b--')
for asset_index in range(n_assets):
    ax.scatter(x=np.sqrt(cov_mat.iloc[asset_index, asset_index]),
                y=avg_returns[asset_index],
                #marker=MARKS[asset_index],
                s=150,
                color='black',
                label=RISKY_ASSETS[asset_index])
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
max_sharpe_ind = np.argmax(portf_results_df.sharpe_ratio)
max_sharpe_portf = portf_results_df.loc[max_sharpe_ind]

min_vol_ind = np.argmin(portf_results_df.volatility)
min_vol_portf = portf_results_df.loc[min_vol_ind]

In [ ]:
print('Maximum Sharpe Ratio portfolio ----')
print('Performance')
for index, value in max_sharpe_portf.items():
    print(f'{index}: {100 * value:.2f}% ', end="", flush=True)
print('\nWeights')
for x, y in zip(RISKY_ASSETS, weights[np.argmax(portf_results_df.sharpe_ratio)]):
    print(f'{x}: {100*y:.2f}% ', end="", flush=True)

In [ ]:
print('Minimum Volatility portfolio ----')
print('Performance')
for index, value in min_vol_portf.items():
    print(f'{index}: {100 * value:.2f}% ', end="", flush=True)
print('\nWeights')
for x, y in zip(RISKY_ASSETS, weights[np.argmin(portf_results_df.volatility)]):
    print(f'{x}: {100*y:.2f}% ', end="", flush=True)

In [ ]:
fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter', x='volatility',
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black',
                      ax=ax)
ax.scatter(x=max_sharpe_portf.volatility,
           y=max_sharpe_portf.returns,
           c='black', marker='*',
           s=200, label='Max Sharpe Ratio')
ax.scatter(x=min_vol_portf.volatility,
           y=min_vol_portf.returns,
           c='black', marker='P',
           s=200, label='Minimum Volatility')
ax.set(xlabel='Volatility', ylabel='Expected Returns',
       title='Efficient Frontier')
ax.legend()

plt.tight_layout()
plt.show()